In [1]:
!nvidia-smi

Fri Dec  5 11:39:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Training

## Preparation

In [1]:
!git clone https://github.com/upiupiup/projectakhir-dl-sam-pytorch.git

Cloning into 'projectakhir-dl-sam-pytorch'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 247 (delta 89), reused 74 (delta 71), pack-reused 140 (from 3)
Receiving objects: 100% (247/247), 162.79 MiB | 39.45 MiB/s, done.
Resolving deltas: 100% (111/111), done.
Updating files: 100% (43/43), done.


In [2]:
%cd projectakhir-dl-sam-pytorch

/kaggle/working/projectakhir-dl-sam-pytorch


In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torchvision

In [5]:
!ls

example  img  LICENSE  README.md  requirements.txt  sam.py


Helper Function

In [16]:
import os

PROJECT = "/kaggle/working/projectakhir-dl-sam-pytorch"
datasets = ["cifar10", "svhn"]
optimizers = ["sgd", "sam_sgd"]
seed = 0
epochs = 20

def run_once(dataset, optimizer, seed, epochs):
    out_dir = f"/kaggle/working/results/{dataset}_{optimizer}_seed{seed}"
    os.makedirs(out_dir, exist_ok=True)

    cmd = f"""
PYTHONPATH={PROJECT} \
python {PROJECT}/example/train.py \
    --dataset {dataset} \
    --model resnet18 \
    --optimizer {optimizer} \
    --epochs {epochs} \
    --batch_size 512 \
    --seed {seed} \
    --output_dir {out_dir}
    """
    print(f"\n=== RUN: dataset={dataset}, opt={optimizer}, seed={seed} ===")
    os.system(cmd)


## Sanity Check

In [9]:
!PYTHONPATH=/kaggle/working/projectakhir-dl-sam-pytorch \
  python example/train.py \
    --dataset cifar10 \
    --model resnet18 \
    --optimizer sgd \
    --epochs 10 \
    --batch_size 128 \
    --output_dir /kaggle/working/results

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.5974  │     26.48 %  ┃   1.000e-01  │   01:00 min  ┃┈██████████████████████████▓┈┨      1.2626  │     38.60 %  ┃
┃           1  ┃      1.1674  │     43.75 %  ┃   1.000e-01  │   00:54 min  ┃┈██████████████████████████▓┈┨      0.9967  │     53.01 %  ┃
┃           2  ┃      0.9425  │     55.61 %  ┃   1.000e-01  │   00:54 min  ┃┈██████████████████████████▓┈┨      0.9666  │     55.44 %  ┃
┃           3  ┃      0.7095  │     67.10 %  ┃   2.000e-02  │   00:54 min  ┃┈██████████████████████████▓┈┨      0.6609  │     69.48 %  ┃
┃           4  ┃      0.

## CIFAR10

### SGD

In [7]:
for opt in ["sgd", "sam_sgd"]:
    run_once("cifar10", opt, seed, epochs=20)


=== RUN: dataset=cifar10, opt=sgd, seed=0 ===


100%|██████████| 170M/170M [00:03<00:00, 54.7MB/s] 


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.7142  │     23.87 %  ┃   1.000e-01  │   00:59 min  ┃┈██████████████████████████▓┈┨      1.3236  │     36.62 %  ┃
┃           1  ┃      1.2787  │     38.02 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      1.1119  │     47.08 %  ┃
┃           2  ┃      1.0988  │     47.27 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      0.9527  │     55.65 %  ┃
┃           3  ┃      0.9402  │     55.64 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      0.9802  │     55.95 %  ┃
┃           4  ┃      0.

In [11]:
%cd /kaggle/working
!zip -r results.zip results/

/kaggle/working
  adding: results/ (stored 0%)
  adding: results/cifar10_sgd_seed0/ (stored 0%)
  adding: results/cifar10_sgd_seed0/checkpoints/ (stored 0%)
  adding: results/cifar10_sgd_seed0/checkpoints/cifar10_resnet18_sgd_lr0.1_wd0.0005_bs128_seed0_best.pt (deflated 7%)
  adding: results/cifar10_sgd_seed0/epochs/ (stored 0%)
  adding: results/cifar10_sgd_seed0/epochs/cifar10_resnet18_sgd_lr0.1_wd0.0005_bs128_seed0_20251205-151605.csv (deflated 59%)
  adding: results/cifar10_sgd_seed0/summary.csv (deflated 36%)
  adding: results/cifar10_sam_sgd_seed0/ (stored 0%)
  adding: results/cifar10_sam_sgd_seed0/checkpoints/ (stored 0%)
  adding: results/cifar10_sam_sgd_seed0/checkpoints/cifar10_resnet18_sam_sgd_rho0.05_adaptive0_lr0.1_wd0.0005_bs128_seed0_best.pt (deflated 7%)
  adding: results/cifar10_sam_sgd_seed0/epochs/ (stored 0%)
  adding: results/cifar10_sam_sgd_seed0/epochs/cifar10_resnet18_sam_sgd_rho0.05_adaptive0_lr0.1_wd0.0005_bs128_seed0_20251205-153623.csv (deflated 58%)
  addi

### AdamW

In [12]:
for opt in ["adamw", "sam_adamw"]:
    run_once("cifar10", opt, seed, epochs=20)


=== RUN: dataset=cifar10, opt=adamw, seed=0 ===


100%|██████████| 170M/170M [00:01<00:00, 95.7MB/s] 


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.7900  │     22.09 %  ┃   1.000e-01  │   00:58 min  ┃┈██████████████████████████▓┈┨      1.3523  │     33.81 %  ┃
┃           1  ┃      1.3409  │     34.50 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      1.2248  │     41.19 %  ┃
┃           2  ┃      1.2064  │     41.65 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      1.1820  │     44.91 %  ┃
┃           3  ┃      1.0918  │     48.09 %  ┃   1.000e-01  │   00:55 min  ┃┈██████████████████████████▓┈┨      1.0006  │     53.35 %  ┃
┃           4  ┃      0.

In [15]:
%cd /kaggle/working
!zip -r adamw_results.zip results/cifar10_adamw_seed0 results/cifar10_sam_adamw_seed0

/kaggle/working
  adding: results/cifar10_adamw_seed0/ (stored 0%)
  adding: results/cifar10_adamw_seed0/checkpoints/ (stored 0%)
  adding: results/cifar10_adamw_seed0/checkpoints/cifar10_resnet18_adamw_lr0.1_wd0.0005_bs128_seed0_best.pt (deflated 26%)
  adding: results/cifar10_adamw_seed0/epochs/ (stored 0%)
  adding: results/cifar10_adamw_seed0/epochs/cifar10_resnet18_adamw_lr0.1_wd0.0005_bs128_seed0_20251205-162007.csv (deflated 58%)
  adding: results/cifar10_adamw_seed0/summary.csv (deflated 37%)
  adding: results/cifar10_sam_adamw_seed0/ (stored 0%)
  adding: results/cifar10_sam_adamw_seed0/checkpoints/ (stored 0%)
  adding: results/cifar10_sam_adamw_seed0/checkpoints/cifar10_resnet18_sam_adamw_rho0.05_adaptive0_lr0.1_wd0.0005_bs128_seed0_best.pt (deflated 8%)
  adding: results/cifar10_sam_adamw_seed0/epochs/ (stored 0%)
  adding: results/cifar10_sam_adamw_seed0/epochs/cifar10_resnet18_sam_adamw_rho0.05_adaptive0_lr0.1_wd0.0005_bs128_seed0_20251205-164019.csv (deflated 58%)
  addi

## SVHN

### SGD

In [ ]:
for opt in ["sgd", "sam_sgd"]:
    run_once("svhn", opt, seed, epochs=20)

### AdamW